In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
df = pd.read_csv('/kaggle/input/traincsv/train.csv')

In [3]:
from tensorflow.keras.layers import TextVectorization

In [4]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [5]:
MAX_FEATURES = 200000 #number of words in vocabulary
vectorizer = TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=1800, output_mode='int')
vectorizer.adapt(X.values)

2023-02-05 12:47:08.243627: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 12:47:08.341328: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 12:47:08.342456: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 12:47:08.344611: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
#vectorizer.get_vocabulary()
#MCSHBAP here, pipeline steps
vectorized_text = vectorizer(X.values)
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

2023-02-05 12:47:35.019606: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2297822400 exceeds 10% of free system memory.


In [7]:
batch_X, batch_y = dataset.as_numpy_iterator().next()
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2)) #skip 70% and take 20%
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1)) #skip 90% and take 10%
train_generator = train.as_numpy_iterator()

2023-02-05 12:47:36.751355: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2297822400 exceeds 10% of free system memory.
2023-02-05 12:47:39.184433: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2297822400 exceeds 10% of free system memory.


In [8]:
#Sequential model part
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [9]:
# y.shape
model = Sequential()
model.add(Embedding(MAX_FEATURES+1, 32)) #Create embedding layer
model.add(Bidirectional(LSTM(32, activation='tanh'))) #Create LSTM layer with 32 units
model.add(Dense(128, activation='relu')) #feature extractor
model.add(Dense(256, activation='relu')) #feature extractor
model.add(Dense(128, activation='relu')) #feature extractor
model.add(Dense(6, activation='sigmoid')) #final layer
model.compile(loss='BinaryCrossentropy', optimizer='Adam')
# model.summary()

In [10]:
history = model.fit(train, epochs=1, validation_data=val)

2023-02-05 12:47:41.777141: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2297822400 exceeds 10% of free system memory.
2023-02-05 12:47:43.707681: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2297822400 exceeds 10% of free system memory.
2023-02-05 12:47:50.281902: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


6981/6981 [==============================] - 787s 112ms/step - loss: 0.0622 - val_loss: 0.0460


In [16]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [17]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [18]:
for batch in test.as_numpy_iterator():
    X_true, y_true = batch
    yhat = model.predict(X_true)
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [20]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

Precision: 0.8378474712371826, Recall: 0.6749927997589111, Accuracy: 0.4864593744277954


In [22]:
model.save('toxicity.h5')